In [ ]:
!pip install parsivar
!pip install stopwordsiso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492971 sha256=25743f48b4f6250843f46fb922df9e84aba41a4041eddc71a43403509b9fa052
  Stored in directory: /root/.cache/pip/wheels/54/0a/38/7d0b1aabbd644340a94fb8685fd20d9f35814d735973d07f40
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449921 sha256=2c8ea937fe280877553c28415ee1e4e9128821499968c983a41dfe03dcc7bd57
  Stored in directory: /root/.cache/pip/wheels/23/18/48/8fd6ec11da38406b309470566d6f099c04805d2ec61d7829e7
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Suc

In [ ]:
import json
import parsivar
from parsivar import Normalizer
from parsivar import Tokenizer
from parsivar import FindStems
from stopwordsiso import stopwords
import re
from string import punctuation
import pandas as pd
import pickle
import numpy as np
from numpy.linalg import norm
from numpy import array

In [ ]:
# loading the 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# vars
IR_data_news = pd.read_json('/content/drive/MyDrive/IR_data_news_12k.json')
positional_index_path = ('/content/drive/MyDrive/positional_index.txt')
preprocessed_news_path = ('/content/drive/MyDrive/preprocessed_news.txt')
IR_data_news.shape

(6, 12202)

In [ ]:
# loading positional index from google drive
with open(positional_index_path, "rb") as myFile:
    saved_positional_index = pickle.load(myFile)
with open(preprocessed_news_path, "rb") as myFile:
    saved_preprocessed_news = pickle.load(myFile)

In [ ]:
def remove_punctuation(text):
    new_text = re.sub(f'[{punctuation}؟،٪×÷.»«()+:إ؛]', '', text)
    return new_text
 
def normalization(text):
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(text)
    return normalized_text
    
def tokenizer(text):
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize_words(text)
    return tokens

def delete_stopWords(tokens):
    stop_words = stopwords('fa')
    for i in tokens:
        if i in stop_words:
            tokens.remove(i)
    return tokens
    
def stemming(tokens):
    new_tokens = []
    my_stemmer = FindStems()
    for t in tokens:
        stems = my_stemmer.convert_to_stem(t)
        stems = stems.split("&")
        if len(stems) == 2:
          new_tokens.append(stems[1])
        else:
          new_tokens.append(stems[0])
    return new_tokens

def preprocess_text(text):
   text = remove_punctuation(text)
   normalized_text = normalization(text)
   tokens = tokenizer(normalized_text)
   tokens = delete_stopWords(tokens)
   stems = stemming(tokens)
   return stems

In [ ]:
 # calculating tf-idfs
documents_weights = {}
N = len(saved_preprocessed_news)
for doc_id, doc_content in enumerate(saved_preprocessed_news):
  documents_weights[doc_id] = {}
  for token in doc_content:
    f_td = saved_positional_index[token][1][doc_id][0] # number of occurences of term t in doc d
    tf = 1 + np.log10(f_td) 
    n_t = len(saved_positional_index[token][1]) # number of docs that have term t
    idf = np.log10(N/n_t)
    token_weight = tf*idf
    documents_weights[doc_id][token] = token_weight

In [ ]:
documents_weights[5000]

{'گزارش': 0.06598686528979414,
 'خبرگزاری': 0.007249774608743844,
 'فارس': 0.004795719153417859,
 'چهارشنبه': 1.3222548882660379,
 'شب': 1.0765553869442086,
 'چارچوب': 1.3025274413836339,
 'دیدار': 0.6696241488334242,
 'نیمه': 1.1278671374344014,
 'نهایی': 1.0280049961993634,
 'سوپرکاپ': 2.5301285198890815,
 'اسپانیا': 1.8008737116485949,
 'بارسلونا': 2.486453717075493,
 'رئال': 2.4591514856652594,
 'مادرید': 2.201315873162769,
 'ریاض': 2.5549521036141134,
 'مصاف': 1.1014538942408754,
 'خواه': 0.6763289440135081,
 'ژاوی': 2.686941428080667,
 'لیست': 1.890347879741773,
 'بازیکن': 0.713760571162582,
 'آماده': 1.5828497624019542,
 'بازی': 0.6918807928179967,
 'حساس': 1.4015856590119562,
 'اعلام': 1.3784274799870533,
 'که': 0.30875894820441563,
 'آنسو': 4.015539337421733,
 'فاتی': 4.218631861942555,
 'چشم': 1.4975992950621615,
 'خور': 1.3018137280234934,
 'باشگاه': 0.6976196071828451,
 'فرنکی': 4.141622171225401,
 'دی': 1.8626571443623743,
 'یونگ': 3.876014951001258,
 'آرائخو': 3.715718378

In [ ]:
def calculate_query_tfidf(query):
  query_weights = {}
  query_tokens = preprocess_text(query)
  print(query_tokens)
  for token in query_tokens:
    if token in saved_positional_index.keys():
      f_td = query_tokens.count(token)
      tf = 1 + np.log10(f_td)
      n_t = len(saved_positional_index[token][1])
      idf = np.log10(N/n_t)
      token_weight = tf*idf
      query_weights[token] = token_weight
  return query_weights
print(calculate_query_tfidf('اینجا تهران یعنی گل فوتبال خانه'))

['تهران', 'گل', 'فوتبال', 'خانه']
{'تهران': 0.8304352939339668, 'گل': 0.7515753310390771, 'فوتبال': 0.5672595568347096, 'خانه': 1.2552013267893054}


In [ ]:
def ranked_retrival(query, saved_positional_index, documents_weights, saved_preprocessed_news):
  scores = {}
  for i in range(len(saved_preprocessed_news)):
    scores[i] = 0
  query_weights = calculate_query_tfidf(query)
  query_tokens = list(query_weights.keys())
  print(query_tokens)
  print("******")
  for term in query_tokens:
    w_tq = query_weights[term]
    term_postings_list = saved_positional_index[term][1].keys()
    for doc_id in term_postings_list:
      w_td = documents_weights[doc_id][term]
      scores[int(doc_id)] += w_tq * w_td
  for d, content in enumerate(saved_preprocessed_news):
    doc_weight_length = array(list(documents_weights[d].values()))
    weight_norm_l2 = norm(doc_weight_length)
    scores[d] /= weight_norm_l2
  final = sorted(scores.items(), key=lambda item: item[1], reverse=True)
  for i in final[:15]:
    print(IR_data_news[i[0]]['title'])
    print(IR_data_news[i[0]]['url'])
  return final[:15]

In [ ]:
ranked_retrival('سلام', saved_positional_index, documents_weights, saved_preprocessed_news)

['سلام']
['سلام']
******
حضور دختر شهید سپهبد سلیمانی در جایگاه سخنرانی سردار سلامی
https://www.farsnews.ir/news/14000813000237/حضور-دختر-شهید-سپهبد-سلیمانی-در-جایگاه-سخنرانی-سردار-سلامی
سجادی: این صعود را به ملت بزرگ ایران تبریک می‌گویم/امیدوارم از مرحله گروهی جام جهانی صعود کنیم
https://www.farsnews.ir/news/14001107000673/سجادی-این-صعود-را-به-ملت-بزرگ-ایران-تبریک-می‌گویم-امیدوارم-از-مرحله
فرمانده سپاه واکسن «نورا» را دریافت کرد
https://www.farsnews.ir/news/14001012000166/فرمانده-سپاه-واکسن-نورا-را-دریافت-کرد
عیادت نماینده رهبر انقلاب از آیت‌الله علوی‌گرگانی
https://www.farsnews.ir/news/14001202000692/عیادت-نماینده-رهبر-انقلاب-از-آیت‌الله-علوی‌گرگانی
ورزشکار کویتی دست رد به سینه صهیونیست‌ها زد+عکس
https://www.farsnews.ir/news/14001102000416/ورزشکار-کویتی-دست-رد-به-سینه-صهیونیست‌ها-زد-عکس
فرماندهان ارشد نیروهای مسلح با آرمان‌های امام خمینی(ره) تجدید میثاق کردند
https://www.farsnews.ir/news/14001118000270/فرماندهان-ارشد-نیروهای-مسلح-با-آرمان‌های-امام-خمینی-ره-تجدید-میثاق
برتری پرگل تراک

[(11486, 0.3940041251632665),
 (3509, 0.32726617697712546),
 (9185, 0.27069489194428337),
 (7541, 0.26958545730951605),
 (4005, 0.26650823217965663),
 (7920, 0.24752540280801194),
 (551, 0.23867014530768302),
 (4592, 0.2378765244345868),
 (1382, 0.23660655256806176),
 (5800, 0.23302375844943277),
 (1963, 0.2322879421537839),
 (10239, 0.23213128524834856),
 (9024, 0.2264426814947048),
 (11499, 0.2191627659099071),
 (11851, 0.21642402937500696)]

In [ ]:
ranked_retrival('انرژي هسته‌ سانتریفیوژ‎', saved_positional_index, documents_weights, saved_preprocessed_news)

['انرژی', 'هسته', 'سانتریفیوژ']
['انرژی', 'هسته', 'سانتریفیوژ']
******
ایران اکنون دست بالا را در مذاکرات دارد/ دلیل تخریب دستاوردهای دولت از سوی غربگرایان
https://www.farsnews.ir/news/14000920000546/ایران-اکنون-دست-بالا-را-در-مذاکرات-دارد-دلیل-تخریب-دستاوردهای-دولت-از
مرکز پژوهش‌های مجلس: مذاکرات وین به توافقی زودهنگام منجر نمی‌شود
https://www.farsnews.ir/news/14000924000773/مرکز-پژوهش‌های-مجلس-مذاکرات-وین-به-توافقی-زودهنگام-منجر-نمی‌شود
سازمان‌های مدعی‌ حقوق بشر چگونه مقابل درنده‌خویی حاکمان سعودی سکوت می‌کنند؟
https://www.farsnews.ir/news/14001223000865/سازمان‌های-مدعی‌-حقوق-بشر-چگونه-مقابل-درنده‌خویی-حاکمان-سعودی-سکوت
همایش «فخر دماوند»| عباسی: شهید فخری‌زاده همواره از متخصصین حمایت می‌کرد
https://www.farsnews.ir/news/14000913000631/همایش-فخر-دماوند|-عباسی-شهید-فخری‌زاده-همواره-از-متخصصین-حمایت-می‌کرد
دستگاه دیپلماسی دزدی دریایی آمریکا را در مجامع بین‌المللی پیگیری کند/ تقدیر از اقدام مقتدرانه سپاه
https://www.farsnews.ir/news/14000812000699/دستگاه-دیپلماسی-دزدی-دریایی-آمریکا-را-در

[(9896, 0.4946631406488581),
 (9742, 0.45217847692387086),
 (6891, 0.39144269231468887),
 (10257, 0.3860054906668984),
 (11523, 0.3664306337572922),
 (9882, 0.3451980841764148),
 (7795, 0.32990325725339614),
 (11694, 0.3029370357195009),
 (9720, 0.30080695230843185),
 (9702, 0.29757034744410726),
 (6956, 0.2974506519125601),
 (8072, 0.2911857643631645),
 (9591, 0.2850238544149267),
 (10217, 0.28039726223006384),
 (11584, 0.26662162588593646)]

In [ ]:
ranked_retrival('سلام تراکنش رفتنی', saved_positional_index, documents_weights, saved_preprocessed_news)

['سلام', 'تراکنش', 'رفتن']
['سلام', 'تراکنش', 'رفتن']
******
نظر سازمان خصوصی‌سازی در مورد ساختمان استقلال و پرسپولیس
https://www.farsnews.ir/news/14001207001106/نظر-سازمان-خصوصی‌سازی-در-مورد-ساختمان-استقلال-و-پرسپولیس
جزئیات جلسه کمیته اقتصاد دیجیتال مجلس درباره «اینماد»/ تأکید بر تدوین مقررات تبادل رمزارز
https://www.farsnews.ir/news/14000917000347/جزئیات-جلسه-کمیته-اقتصاد-دیجیتال-مجلس-درباره-اینماد-تأکید-بر-تدوین
میرکاظمی: برای بررسی دقیق‌تر بودجه فرهنگی، کمیته‌ای تشکیل شود
https://www.farsnews.ir/news/14001105000824/میرکاظمی-برای-بررسی-دقیق‌تر-بودجه-فرهنگی-کمیته‌ای-تشکیل-شود
حضور دختر شهید سپهبد سلیمانی در جایگاه سخنرانی سردار سلامی
https://www.farsnews.ir/news/14000813000237/حضور-دختر-شهید-سپهبد-سلیمانی-در-جایگاه-سخنرانی-سردار-سلامی
مخالفت مجلس با افزایش هزینه تراکنش بانکی در سال ۱۴۰۱
https://www.farsnews.ir/news/14001209000666/مخالفت-مجلس-با-افزایش-هزینه-تراکنش-بانکی-در-سال-۱۴۰۱
سجادی: این صعود را به ملت بزرگ ایران تبریک می‌گویم/امیدوارم از مرحله گروهی جام جهانی صعود کنیم
https:/

[(1255, 0.5588467520926912),
 (10046, 0.5081083552061358),
 (8372, 0.4944402382668654),
 (11486, 0.3940041251632665),
 (7342, 0.3823040103471947),
 (3509, 0.32726617697712546),
 (244, 0.31746371453486566),
 (7096, 0.29958580393648654),
 (9185, 0.27069489194428337),
 (7541, 0.26958545730951605),
 (4005, 0.26650823217965663),
 (342, 0.2513921940591628),
 (7920, 0.24752540280801194),
 (2604, 0.24231727035777048),
 (7056, 0.2415674944288764)]

In [ ]:
# # champions list with tf-idf
# champions_list = {}
# for term in saved_positional_index.keys():
#   docs_weights = []
#   for doc_id in documents_weights:
#       if term in documents_weights[doc_id].keys():
#         docs_weights.append((doc_id, documents_weights[doc_id][term]))
#   ranked_weights = sorted(docs_weights, key=lambda item: item[1], reverse=True)[:15]
#   champions_list[term] = ranked_weights
# print(champions_list)

# IOPub data rate exceeded

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# champions list with frequencies
def make_champions_list(saved_positional_index, k):
  champions_list = {}
  for term in saved_positional_index:    
    postings = saved_positional_index[term][1] 
    #print(list(postings.values())[0])  
    sorted_docs = sorted(postings.items(), key=lambda x: x[1][0], reverse=True)  
    best_docs =  sorted_docs[:min(k,len(sorted_docs))]  
    champions_list[term] = [item[0] for item in best_docs]
  print(champions_list['سلام'])
  return champions_list 

In [ ]:
champions_lists = make_champions_list(saved_positional_index, 15)
print(type(champions_lists))

[9018, 8973, 10824, 8898, 11495, 9701, 10745, 8483, 8882, 8752, 11220, 11489, 9179, 11076, 1382]
<class 'dict'>


In [ ]:
# save champions list :
with open("champions_list.txt", "wb") as myFile2:
    pickle.dump(champions_lists, myFile2)

In [ ]:
def intersect_results(all_champs):
  result = all_champs[0]
  for doc_id in all_champs[0]:
    #print(doc_id)
    if len(all_champs) > 1:
      for champion_list in all_champs[1:]:
        if doc_id not in champion_list:
          if doc_id in result:
            result.remove(doc_id)
  print(result)
  return result

In [ ]:
def champions_list_retrival(query):
  ranked_docs = []
  query_champions_lists = []
  query_tokens = preprocess_text(query)
  for token in query_tokens:
    if token in champions_lists.keys():
      for i in champions_lists[token]:
        query_champions_lists.append(i)
  ranked_docs = sorted(set(query_champions_lists), key=lambda ele: query_champions_lists.count(ele), reverse=True)
  return ranked_docs[:15]

In [ ]:
champions_lists['گزارش']

[11604,
 11449,
 9500,
 11163,
 11774,
 11686,
 10139,
 8960,
 7639,
 11257,
 10663,
 12107,
 10761,
 11224,
 7006]

In [ ]:
champions_lists['فارس']

[163,
 6404,
 11697,
 1322,
 1633,
 6755,
 7584,
 821,
 2388,
 7744,
 5395,
 525,
 8680,
 10183,
 9687]

In [ ]:
champions_list_retrival(' گزارش فارس ')

[8960,
 6404,
 10761,
 525,
 5395,
 10139,
 11163,
 9500,
 7584,
 163,
 11686,
 10663,
 1322,
 11697,
 821]

In [ ]:
ranked_retrival('حقوق کارگران', saved_positional_index, documents_weights, saved_preprocessed_news)

['حقوق', 'کارگر']
['حقوق', 'کارگر']
******
رئیس‌جمهور: کسی که به کارگر ظلم می‌کند، صلاحیت ندارد پیمانکار دولت باشد
https://www.farsnews.ir/news/14001215000943/رئیس‌جمهور-کسی-که-به-کارگر-ظلم-می‌کند-صلاحیت-ندارد-پیمانکار-دولت-باشد
بدری: حقوق کارگر در ادارات و شرکت‌های دولتی رعایت نمی‌شود
https://www.farsnews.ir/news/14000812000303/بدری-حقوق-کارگر-در-ادارات-و-شرکت‌های-دولتی-رعایت-نمی‌شود
مصوبه کمیسیون اجتماعی برای تعیین حق بیمه کارگران ساختمانی
https://www.farsnews.ir/news/14000816000677/مصوبه-کمیسیون-اجتماعی-برای-تعیین-حق-بیمه-کارگران-ساختمانی
مجلس تبعات مثبت و منفی افزایش حقوق و دستمزد کارگران را بررسی کند
https://www.farsnews.ir/news/14001224000134/مجلس-تبعات-مثبت-و-منفی-افزایش-حقوق-و-دستمزد-کارگران-را-بررسی-کند
شوراهای کارگری بدون پشت و پناه هستند
https://www.farsnews.ir/news/14000809000321/شوراهای-کارگری-بدون-پشت-و-پناه-هستند
اعلام ورزشگاه یک مسابقه از هفته دوازدهم لیگ یک/ خوشه طلایی بدون زمین!
https://www.farsnews.ir/news/14001005000516/اعلام-ورزشگاه-یک-مسابقه-از-هفته-دوازدهم-لیگ-یک

[(7177, 0.6996941965045742),
 (11549, 0.5858130931539991),
 (11359, 0.5043483296814466),
 (6869, 0.4898205812996476),
 (11731, 0.46882116852662337),
 (6098, 0.443268594731831),
 (2688, 0.43059069583436976),
 (9642, 0.4099638426468446),
 (8412, 0.4054598969225917),
 (12065, 0.3965292913557341),
 (10355, 0.3891589419956344),
 (6031, 0.38596175344511685),
 (10201, 0.36221374174973836),
 (10284, 0.36142221000955044),
 (2991, 0.3590840743918076)]

In [ ]:
ranked_retrival(' پول تلاش کارگران', saved_positional_index, documents_weights, saved_preprocessed_news)

['پول', 'تلاش', 'کارگر']
['پول', 'تلاش', 'کارگر']
******
رئیس‌جمهور: کسی که به کارگر ظلم می‌کند، صلاحیت ندارد پیمانکار دولت باشد
https://www.farsnews.ir/news/14001215000943/رئیس‌جمهور-کسی-که-به-کارگر-ظلم-می‌کند-صلاحیت-ندارد-پیمانکار-دولت-باشد
بدری: حقوق کارگر در ادارات و شرکت‌های دولتی رعایت نمی‌شود
https://www.farsnews.ir/news/14000812000303/بدری-حقوق-کارگر-در-ادارات-و-شرکت‌های-دولتی-رعایت-نمی‌شود
اعلام ورزشگاه یک مسابقه از هفته دوازدهم لیگ یک/ خوشه طلایی بدون زمین!
https://www.farsnews.ir/news/14001005000516/اعلام-ورزشگاه-یک-مسابقه-از-هفته-دوازدهم-لیگ-یک-خوشه-طلایی-بدون-زمین
شکست شاگردان قلعه‌نویی مقابل ذوب آهن
https://www.farsnews.ir/news/14001118000767/شکست-شاگردان-قلعه‌نویی-مقابل-ذوب-آهن
رئیس جدید سازمان ثبت احوال کشور منصوب شد
https://www.farsnews.ir/news/14000928000785/رئیس-جدید-سازمان-ثبت-احوال-کشور-منصوب-شد
مصوبه کمیسیون اجتماعی برای تعیین حق بیمه کارگران ساختمانی
https://www.farsnews.ir/news/14000816000677/مصوبه-کمیسیون-اجتماعی-برای-تعیین-حق-بیمه-کارگران-ساختمانی
واکنش عضو کم

[(7177, 0.5134598825443585),
 (11549, 0.4501749203787773),
 (6098, 0.443268594731831),
 (2688, 0.43059069583436976),
 (9642, 0.4099638426468446),
 (11359, 0.4095110745460563),
 (11904, 0.39870102373257715),
 (12065, 0.3965292913557341),
 (6031, 0.38596175344511685),
 (11731, 0.3832794273922321),
 (2991, 0.3590840743918076),
 (1748, 0.3552148396626751),
 (10001, 0.3521852149988085),
 (6869, 0.34694857758945985),
 (11321, 0.3453690997725532)]

In [ ]:
ranked_retrival('ژولیده', saved_positional_index, documents_weights, saved_preprocessed_news)

['ژولیده']
['ژولیده']
******
آغاز اردوی تیم‌ملی فوتبال هفت نفره از اول اسفند
https://www.farsnews.ir/news/14001123000537/آغاز-اردوی-تیم‌ملی-فوتبال-هفت-نفره-از-اول-اسفند
اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا
https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا
سجادی :حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون  و سازمان لیگ است
https://www.farsnews.ir/news/14001224000982/سجادی-حضور-تماشاگران-در-لیگ-برتر-فوتبال-تابع-نظر-فدراسیون-و-سازمان
محل برگزاری نشست‌های خبری سرخابی‌ها؛ مجیدی در سازمان لیگ، گل‌محمدی در تمرین پرسپولیس
https://www.farsnews.ir/news/14001224000971/محل-برگزاری-نشست‌های-خبری-سرخابی‌ها-مجیدی-در-سازمان-لیگ-گل‌محمدی-در
ماجدی در نشست با صالحی امیری: امیدوارم در این دوره تیم  المپیک موفق شود
https://www.farsnews.ir/news/14001224000964/ماجدی-در-نشست-با-صالحی-امیری-امیدوارم-در-این-دوره-تیم-المپیک-موفق
لیگ‌برتر بسکتبال|‌ نخستین پیروزی شهرداری گرگان و ذوب‌آهن در پلی‌آف
https://www.farsnews.ir/news/14001224000947/لیگ‌برتر-بسکتب

[(2341, 0.8355763517172049),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0)]

In [ ]:
ranked_retrival('روند پیچیده ', saved_positional_index, documents_weights, saved_preprocessed_news)

['روند', 'پیچیده']
['روند', 'پیچیده']
******
مدیرعامل تراکتور:مشکلی بابت کارت بازی بازیکنان جدید نداریم/نگران 7کرونایی هستیم
https://www.farsnews.ir/news/14001123000396/مدیرعامل-تراکتور-مشکلی-بابت-کارت-بازی-بازیکنان-جدید-نداریم-نگران
اعلام آخرین وضعیت صیادمنش/آینده مهاجم تیم ملی پیچیده شد+عکس
https://www.farsnews.ir/news/14001007000079/اعلام-آخرین-وضعیت-صیادمنش-آینده-مهاجم-تیم-ملی-پیچیده-شد-عکس
حزب کارگزاران دولت سایه تشکیل می دهد
https://www.farsnews.ir/news/14000910000907/حزب-کارگزاران-دولت-سایه-تشکیل-می-دهد
اقدام متفاوت کمیسیون تلفیق/ جداول بودجه امروز برای اولین بار شفاف می‌شود
https://www.farsnews.ir/news/14001214000859/اقدام-متفاوت-کمیسیون-تلفیق-جداول-بودجه-امروز-برای-اولین-بار-شفاف
المپیک زمستانی پکن| نمونه B تست دوپینگ ساوه‌شمشکی هم مثبت شد
https://www.farsnews.ir/news/14001123000260/المپیک-زمستانی-پکن|-نمونه-B-تست-دوپینگ-ساوه‌شمشکی-هم-مثبت-شد
نویدکیا: آلومینیوم اراک  لیگ را شوکه کرده است/سپاهان برای ماندن در بالای جدول امتیاز نیاز دارد
https://www.farsnews.ir/news/140010220008

[(2353, 0.33854255764231894),
 (5986, 0.31629015901098806),
 (10322, 0.28436889074617827),
 (7215, 0.2783345277937334),
 (2369, 0.2691320744459453),
 (4808, 0.2662317091096952),
 (9200, 0.23287397604706508),
 (11023, 0.2326333175324468),
 (5558, 0.23211542066761992),
 (506, 0.22713845971647886),
 (12085, 0.2266708789076457),
 (8351, 0.2229219440765728),
 (10354, 0.21585110917893874),
 (1943, 0.21401516234266663),
 (9587, 0.21294573989195462)]